In [ ]:
"""
main.ipynb

Main file for recreating lottery ticket experiments done in randomly initialized dense neural networks.

Authors: Jordan Bourdeau, Casey Forey
Date Created: 3/8/24
"""

In [178]:
"""
Imports.
"""

%load_ext tensorboard

import datetime
from matplotlib import pyplot as plt
import numpy as np
import os
import random
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D
from tensorflow.keras import datasets
from tensorflow.keras.utils import to_categorical
import tensorflow_model_optimization as tfmot

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [127]:
# Constants for where to output model data
MODEL_DIRECTORY: str = 'models/'
CHECKPOINT_DIRECTORY: str = 'checkpoints/'
FIT_DIRECTORY: str = 'logs/fit/'

def create_path(path: str):
    """
    Helper function to create a path and all its subdirectories.
    :param path: String containing the target path.
    """
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"Directory '{path}' created successfully.")
    else:
        print(f"Directory '{path}' already exists.")

# Create directories if they aren't already created
for directory in [MODEL_DIRECTORY, CHECKPOINT_DIRECTORY, FIT_DIRECTORY]:
    create_path(directory)

Directory 'models/' created successfully.
Directory 'checkpoints/' created successfully.
Directory 'logs/fit/' created successfully.


In [36]:
"""
Load and process the data.
Code Source: https://colab.research.google.com/github/maticvl/dataHacker/blob/master/CNN/LeNet_5_TensorFlow_2_0_datahacker.ipynb#scrollTo=UA2ehjxgF7bY
"""

(X_train, Y_train), (X_test, Y_test) = datasets.mnist.load_data()

# Verify output looks right
print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
print(X_train[0].shape, 'image shape')

# Add a new axis
X_train = X_train[:, :, :, np.newaxis]
X_test = X_test[:, :, :, np.newaxis]

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
print(X_train[0].shape, 'image shape')

# Convert class vectors to binary class matrices.

num_classes = 10
Y_train = to_categorical(Y_train, num_classes)
Y_test = to_categorical(Y_test, num_classes)

# Data normalization
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

x_train shape: (60000, 28, 28)
60000 train samples
10000 test samples
(28, 28) image shape
X_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
(28, 28, 1) image shape


In [97]:
"""
Class definition for the network architecture.
Code Source: https://colab.research.google.com/github/maticvl/dataHacker/blob/master/CNN/LeNet_5_TensorFlow_2_0_datahacker.ipynb#scrollTo=UA2ehjxgF7bY
"""

# LeNet-5 model derived from this paper: http://vision.stanford.edu/cs598_spring07/papers/Lecun98.pdf
class LeNet(Sequential):
    def __init__(self, input_shape: np.array, num_classes: int, **kwargs):
        super().__init__()

        # Convolutional layers  
        self.add(Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=input_shape, padding="same"))
        self.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
        self.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))
        self.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
        self.add(Flatten())

        # Fully connected output layers
        self.add(Dense(120, activation='tanh'))
        self.add(Dense(84, activation='tanh'))
        self.add(Dense(num_classes, activation='softmax'))

        self.compile(optimizer='adam', loss=categorical_crossentropy, metrics=['accuracy'])

In [182]:
"""
Code for creating, training + testing, saving, and loading models.
""" 

def get_model_directory(model_index: int, base_directory: str = "",) -> str:
    """
    Function to return the relative directory where a model would go.

    :param base_directory: Base directory to append model subdirectory to. Defaults to empty string.
    :param model_index: Integer for the index/random seed of the model.

    :returns: Returns expected directory for the model.
    """
    return f'{base_directory}model_{model_index}/'

def get_model_name(model_index: int, pruning_step: int = 0) -> str:
    """
    Function to return the expected name for a model based on its index and pruning step.

    :param model_index:   Integer for the index/random seed of the model.
    :param pruning_step:  Integer for the pruning iteration.

    :returns: Returns expected name for the model.
    """
    return f'model_{model_index}_step_{pruning_step}.keras'

def create_model(feature_shape: tuple[int, ...], num_classes: int, random_seed: int) -> tuple[LeNet, list[tf.keras.callbacks]]:
    """
    Method used for setting the random seed(s) and instantiating a model.

    :param feature_shape:  Shape of the features.
    :param num_classes:    Number of potential classes. 10 for MNIST.
    :param random_seed:    Value used to ensure reproducability.

    :returns: Model and callbacks.
    """
    # Set seeds
    os.environ['PYTHONHASHSEED'] = str(random_seed)
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

    # Initialize the model
    model: LeNet = LeNet(feature_shape, num_classes)

    # Crate the callbacks
    model_name: str = get_model_name(random_seed, 0)
    tensorboard_path: str = get_model_directory(random_seed, FIT_DIRECTORY)
    checkpoint_path: str = get_model_directory(random_seed, CHECKPOINT_DIRECTORY)

    # Create the model checkpoint callback
    callbacks: list[tf.keras.callbacks] = [
        tf.keras.callbacks.TensorBoard(log_dir=tensorboard_path, histogram_freq=1),
        tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True)
    ]

    return model, callbacks

def create_models(X_train: np.array, Y_train: np.array, X_test: np.array, Y_test: np.array, epochs: int, num_models: int):
    """
    Function responsible for training/saving the base, fully parametrized models.

    :param X_train:     Training instances.
    :param X_test:      Testing instances.
    :param Y_train:     Training labels.
    :param Y_test:      Testing labels.
    :param epochs:      Number of epochs to train the model for.
    :param num_models:  Number of models to create.
    """
    assert X_train.shape[0] >= 1, 'Need at least one input to determine feature shape'

    # Extract shape of features and the number of classes
    feature_shape: tuple[int, ...] = X_train[0].shape
    num_classes: int = 10

    # Use index as the random seed input
    for i in range(num_models):
        # Create the model if it does not already exist
        if not os.path.exists(get_model_directory(i, MODEL_DIRECTORY) + get_model_name(i, 0)):
            # Setup and train the model
            model, callbacks = create_model(feature_shape, num_classes, random_seed=i)
            model.fit(X_train, Y_train, epochs=epochs, validation_data=(X_test, Y_test), callbacks=callbacks, verbose=1, use_multiprocessing=True) 
            # Create the model's output directory and save it
            output_directory: str = get_model_directory(i, MODEL_DIRECTORY)
            create_path(output_directory)
            model.save(output_directory + get_model_name(i, 0))

def load_model(feature_shape: tuple[int, ...], num_classes: int, model_index: int, pruning_step: int) -> LeNet:
    """
    Function used to load a single trained model.

    :param feature_shape:     Tuple of integer dimensions for the feature shape.
    :param num_classes:       Number of unique classes for the model.
    :param model_index:       Index of the model which was trained.
    :param pruning_step:      Integer value for the number of pruning steps which had been completed for the model.

    :returns: Model object with weights loaded.
    """
    path: str = get_model_directory(model_index, MODEL_DIRECTORY) + get_model_name(model_index, pruning_step)
    model: LeNet = LeNet(feature_shape, num_classes)
    model.load_weights(path)
    return model

def prune_model(model: LeNet, pruning_percentage: float) -> LeNet:
    """
    Function to prune a given model according to the specified percentage.

    :param model:                 Model to be pruned.
    :param pruning_percentage:    Percentage of weights to prune.

    :returns: Pruned model.
    """
    pruning_schedule = tfmot.sparsity.keras.ConstantSparsity(pruning_percentage, 0)
    # Prune each layer in the model
    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.Dense):
            tfmot.sparsity.keras.prune_low_magnitude(layer, pruning_schedule=pruning_schedule)
    return model

def lottery_ticket_hypothesis(X_train: np.array, Y_train: np.array, X_test: np.array, Y_test: np.array, total_pruning_percentage: float, pruning_steps: int, epochs_per_step: int, num_models: int):
    """
    Function to perform the lottery ticket hypothesis with iterative magnitude pruning.

    :param X_train:                  Training instances.
    :param X_test:                   Testing instances.
    :param Y_train:                  Training labels.
    :param Y_test:                   Testing labels.
    :param total_pruning_percentage: The total percentage of weights to prune.
    :param pruning_steps:            Number of pruning steps.
    :param epochs_per_step:          Number of epochs to train each model for in each step.
    :param num_models:               Number of models to create.

    """
    assert pruning_steps > 0, "Pruning steps should be greater than 0"
    assert total_pruning_percentage > 0 and total_pruning_percentage <= 1, "Total pruning percentage should be between 0 and 1"

    # Create the original models if they don't already exist
    create_models(X_train, Y_train, X_test, Y_test, epochs, num_models)

    feature_shape: tuple[int, ...] = X_train[0].shape
    num_classes: int = 10
    step_pruning_percent: float = total_pruning_percentage / pruning_steps

    # Iterate over each model
    for model_index in range(num_models):
        # Load the model at pruning step 0
        model = load_model(feature_shape, num_classes, model_index, 0)

        # Iterate over each pruning step
        for step in range(pruning_steps):
            output_path: str = get_model_directory(model_index, MODEL_DIRECTORY) + get_model_name(model_index, step + 1)

            # Prune the model
            pruned_model = prune_model(model, step_pruning_percent)

            # Train the pruned model
            pruned_model.fit(X_train, Y_train, epochs=epochs_per_step, validation_data=(X_test, Y_test), verbose=1)

            # Replace the model
            model = pruned_model

            # Save the pruned model with the corresponding pruning percentage
            print(f'Saving model to {output_path}')
            pruned_model.save(output_path)

In [183]:
# Parameters for simulation
num_models: int = 2
epochs: int = 1
total_pruning_percentage: float = 0.99
pruning_steps: int = 2

# Run the simulation
lottery_ticket_hypothesis(X_train, Y_train, X_test, Y_test, total_pruning_percentage, pruning_steps, epochs, num_models)

1875/1875 [==============================] - 7s 4ms/step - loss: 0.0922 - accuracy: 0.9713 - val_loss: 0.0709 - val_accuracy: 0.9777
Saving model to models/model_0/model_0_step_1.keras
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0583 - accuracy: 0.9823 - val_loss: 0.0595 - val_accuracy: 0.9804
Saving model to models/model_0/model_0_step_2.keras
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0855 - accuracy: 0.9732 - val_loss: 0.0704 - val_accuracy: 0.9765
Saving model to models/model_1/model_1_step_1.keras
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0565 - accuracy: 0.9825 - val_loss: 0.0609 - val_accuracy: 0.9793
Saving model to models/model_1/model_1_step_2.keras


In [184]:
feature_shape: tuple = X_train[0].shape
num_classes = 10

unpruned = load_model(feature_shape, num_classes, 0, 0)
pruned_1_step = load_model(feature_shape, num_classes, 0, 1)

In [155]:
print(unpruned.weights[0])

<tf.Variable 'conv2d_120/kernel:0' shape=(5, 5, 1, 6) dtype=float32, numpy=
array([[[[ 0.15055081, -0.04648927,  0.13614608, -0.03882761,
          -0.16479398, -0.09699005]],

        [[ 0.14934656,  0.210729  ,  0.11351886,  0.10385656,
          -0.00950568,  0.09820782]],

        [[ 0.17029496,  0.00887411, -0.01046964,  0.20520319,
          -0.08432572,  0.2863829 ]],

        [[ 0.08597324, -0.11030123,  0.13981798,  0.25078136,
           0.02439029,  0.28530684]],

        [[ 0.23382324,  0.21384415, -0.14387712,  0.15070887,
          -0.06674127,  0.20849589]]],


       [[[ 0.26064548,  0.03713089,  0.01374489,  0.07009594,
          -0.28974587,  0.10976572]],

        [[ 0.23156106,  0.24122684,  0.32346332,  0.213563  ,
           0.02478233, -0.08258878]],

        [[ 0.15418793,  0.22747006,  0.27860987, -0.00909648,
           0.13668007,  0.3282338 ]],

        [[-0.06090546,  0.18914331,  0.23728716,  0.2059265 ,
          -0.01768417,  0.3699549 ]],

        [[-0.

In [185]:
print(pruned_1_step.weights[0])

<tf.Variable 'conv2d_150/kernel:0' shape=(5, 5, 1, 6) dtype=float32, numpy=
array([[[[ 0.18826242, -0.04832909,  0.16186419, -0.02997263,
          -0.20247193, -0.14148208]],

        [[ 0.18260993,  0.21457845,  0.2472481 ,  0.12703685,
          -0.06035525,  0.09343925]],

        [[ 0.20517176, -0.00828617,  0.09555618,  0.24260606,
          -0.13351694,  0.32730266]],

        [[ 0.13112812, -0.15664782,  0.1355724 ,  0.2979849 ,
          -0.01625589,  0.33901107]],

        [[ 0.2896714 ,  0.14561923, -0.21202591,  0.21192057,
          -0.13219354,  0.22461501]]],


       [[[ 0.32934147,  0.04303245,  0.04833424,  0.11902695,
          -0.37549144,  0.07109149]],

        [[ 0.2749379 ,  0.2846999 ,  0.5016536 ,  0.27438253,
          -0.03898301, -0.06852957]],

        [[ 0.18514828,  0.29550833,  0.35536554,  0.04789002,
           0.11973194,  0.41269356]],

        [[-0.01720571,  0.257133  ,  0.20807095,  0.25683478,
          -0.01128332,  0.4826964 ]],

        [[ 0.

In [46]:
# Doesn't work when running but can select "Launch "
%tensorboard --logdir logs/fit

ERROR: Failed to launch TensorBoard (exited with -9).

In [ ]:
"""
Functions for computing metrics on lottery ticket similarities.
""" 

In [ ]:
"""
Code for visualizing results.
"""